## Results

In [1]:
import pandas as pd
path = "./data/data_2023_7_8.csv"
pd.read_csv(path).head()

,Unnamed: 0,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,ignore,symbol
0,1491,1688169600000,240.450,246.300,240.360,243.250,216598.71,1688173199999,52764487.42655,76608,118743.16,28922115.96645,0,BNBUSDT
1,1,1688169600000,30460.20,30527.20,30405.60,30459.10,8790.092,1688173199999,267887525.86960,98338,4269.250,130112872.35110,0,BTCUSDT
2,746,1688169600000,1933.42,1944.69,1929.41,1933.73,126566.533,1688173199999,245167077.95259,123331,63786.188,123575007.63509,0,ETHUSDT
3,2981,1688169600000,108.65,110.00,106.57,108.39,1170358.945,1688173199999,126640122.26958,155040,564158.514,61071727.99589,0,LTCUSDT
4,2236,1688169600000,0.4730,0.4732,0.4666,0.4673,83687231.8,1688173199999,39293437.97834,44238,39954190.8,18760235.31754,0,XRPUSDT


In [1]:
from rl_agent.linearDQNAgent import Agent as linearDQNAgent
from rl_agent.lstmDQNAgent import Agent as lstmDQNAgent

from rl_agent.memory import Transition, ReplayMemory
from function import formatPrice, getState, getTestStockDataVec
from param import *
from tqdm import tqdm
import pandas as pd

## Predict model Linear DQN

In [14]:

window_size = 10
path = "./data/data_2023_7_8.csv"


agent = linearDQNAgent(window_size*5*5) # 10 sample and each sample is 5x5 
data = getTestStockDataVec(path).head(2000)
l = len(data)
time = []
closes = []
buys = []
sells = []
capital = 100000
history_capital = []
history_profit = []


state = getState(data, window_size-1, window_size)

total_profit = 0
agent.inventory = []
# print(l)
# for t in range(window_size-1,l):

for t in range(window_size-1,l-1):


	action = agent.act(state)
 
	close_price = float(data[data['symbol']==BTCUSDT].loc[data.index[t]].close)
	# print(close_price)
 
	closes.append(close_price)
	time.append(data.index[t])
	# print(close_price)b

	# sit
	next_state = getState(data, t + 1, window_size)
	reward = 0

	if action == 1: # buy
		if capital > close_price:
			agent.inventory.append(data.loc[data.index[t]])
			buys.append(close_price)
			sells.append(None)
			capital -= close_price
		else:
			buys.append(None)
			sells.append(None)
		
	elif action == 2: # sell
		if len(agent.inventory) > 0:
      
			bought_price = agent.inventory.pop(0)
		
			# close_price = data[data['symbol']==BTCBUSD].loc[data.index[t]].close
			profit = close_price - float(bought_price[bought_price['symbol']==BTCUSDT].close.values.item())
			reward = max(profit, 0)
			total_profit += profit
			sells.append(close_price)
			buys.append(None)
			capital += close_price
		else:
			buys.append(None)
			sells.append(None)	
		# print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))
		# print('total_profit: ',total_profit)
	elif action == 0:
		buys.append(None)
		sells.append(None)
	
	history_capital.append(capital)
	history_profit.append(total_profit)


	done = True if t%20 == 0 else False
	agent.memory.push(state, action, next_state, reward)
	state = next_state

	if done:
		print("--------------------------------")
		print("Total Profit: " + formatPrice(total_profit))
		print("--------------------------------")

--------------------------------
Total Profit: -$17.20
--------------------------------
--------------------------------
Total Profit: $78.70
--------------------------------
--------------------------------
Total Profit: $207.10
--------------------------------
--------------------------------
Total Profit: $225.30
--------------------------------
--------------------------------
Total Profit: $301.70
--------------------------------
--------------------------------
Total Profit: $292.70
--------------------------------
--------------------------------
Total Profit: $187.80
--------------------------------
--------------------------------
Total Profit: $239.30
--------------------------------
--------------------------------
Total Profit: $233.80
--------------------------------
--------------------------------
Total Profit: $274.50
--------------------------------
--------------------------------
Total Profit: $182.40
--------------------------------
--------------------------------


In [15]:
result_linear = {'time': time,'closes': closes ,'buys': buys, 'sells': sells, "history_capital": history_capital, "history_profit": history_profit}
# Create the DataFrame
df_linear = pd.DataFrame(result_linear)

# Assuming you have a DataFrame named df with a column named 'timestamp' containing Unix timestamps
df_linear['time'] = pd.to_numeric(df_linear['time']) // 1000
df_linear['time'] = pd.to_datetime(df_linear['time'], unit='s')

In [16]:

import plotly.graph_objects as go

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_linear["time"], y=df_linear["closes"],
                    mode='lines',
                    name='closes'))
fig.add_trace(go.Scatter(x=df_linear["time"], y=df_linear["buys"],
                    mode='markers',
                    name='buys'))
fig.add_trace(go.Scatter(x=df_linear["time"], y=df_linear["sells"],
                    mode='markers',
                    name='sells'))
# Set the title
fig.update_layout(title='Stock Prices and Trading Points (Jan 2023) - BTCUSDT - Linear-DQN',
                  xaxis=dict(title='Time'),
                  yaxis=dict(title='Price / Trading Point'))
fig.update_layout(width=800, height=400)
fig.show()

In [17]:
import plotly.graph_objects as go

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_linear["time"], y=df_linear["history_capital"],
                    mode='lines',
                    name='lines'))
# Set the title and axis labels
fig.update_layout(title='Capital Trend Over Time - Linear-DQN',
                  xaxis=dict(title='Time'),
                  yaxis=dict(title='Capital'))
fig.update_layout(width=800, height=400)
fig.show()

In [20]:
import plotly.graph_objects as go

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_linear["time"], y=df_linear["history_profit"],
                    mode='lines',
                    name='lines'))
# Set the title
fig.update_layout(title='History Profit Over Time',
                  xaxis=dict(title='Time'),
                  yaxis=dict(title='History Profit'))
fig.update_layout(width=800, height=400)
fig.show()

## Predict Model LSTM DQN

In [19]:


window_size = 10
path = "./data/data_2023_7_8.csv"


agent = lstmDQNAgent(window_size*5*5) # 10 sample and each sample is 5x5 
data = getTestStockDataVec(path).head(2000)
l = len(data)
time = []
closes = []
buys = []
sells = []
capital = 100000
history_capital = []
history_profit = []


state = getState(data, window_size-1, window_size)

total_profit = 0
agent.inventory = []
# print(l)
# for t in range(window_size-1,l):

for t in range(window_size-1,l-1):


	action = agent.act(state)
 
	close_price = float(data[data['symbol']==BTCUSDT].loc[data.index[t]].close)
	# print(close_price)
 
	closes.append(close_price)
	time.append(data.index[t])
	# print(close_price)b

	# sit
	next_state = getState(data, t + 1, window_size)
	reward = 0

	if action == 1: # buy
		if capital > close_price:
			agent.inventory.append(data.loc[data.index[t]])
			buys.append(close_price)
			sells.append(None)
			capital -= close_price
		else:
			buys.append(None)
			sells.append(None)
		
	elif action == 2: # sell
		if len(agent.inventory) > 0:
      
			bought_price = agent.inventory.pop(0)
		
			# close_price = data[data['symbol']==BTCBUSD].loc[data.index[t]].close
			profit = close_price - float(bought_price[bought_price['symbol']==BTCUSDT].close.values.item())
			reward = max(profit, 0)
			total_profit += profit
			sells.append(close_price)
			buys.append(None)
			capital += close_price
		else:
			buys.append(None)
			sells.append(None)	
		# print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))
		# print('total_profit: ',total_profit)
	elif action == 0:
		buys.append(None)
		sells.append(None)
	
	history_capital.append(capital)
	history_profit.append(total_profit)


	done = True if t%20 == 0 else False
	agent.memory.push(state, action, next_state, reward)
	state = next_state

	if done:
		print("--------------------------------")
		print("Total Profit: " + formatPrice(total_profit))
		print("--------------------------------")

--------------------------------
Total Profit: $0.00
--------------------------------
--------------------------------
Total Profit: -$5.70
--------------------------------
--------------------------------
Total Profit: -$17.70
--------------------------------
--------------------------------
Total Profit: $79.30
--------------------------------
--------------------------------
Total Profit: $216.20
--------------------------------
--------------------------------
Total Profit: $213.10
--------------------------------
--------------------------------
Total Profit: $33.50
--------------------------------
--------------------------------
Total Profit: -$36.20
--------------------------------
--------------------------------
Total Profit: $98.00
--------------------------------
--------------------------------
Total Profit: -$59.90
--------------------------------
--------------------------------
Total Profit: -$98.30
--------------------------------
--------------------------------
Total

In [22]:
result_lstm = {'time': time,'closes': closes ,'buys': buys, 'sells': sells, "history_capital": history_capital, "history_profit": history_profit}
# Create the DataFrame
df_lstm = pd.DataFrame(result_lstm)

In [23]:
# Assuming you have a DataFrame named df with a column named 'timestamp' containing Unix timestamps
df_lstm['time'] = pd.to_numeric(df_lstm['time']) // 1000
df_lstm['time'] = pd.to_datetime(df_lstm['time'], unit='s')

In [24]:
# result_lstm = {'time': time,'closes': closes ,'buys': buys, 'sells': sells, "history_capital": history_capital, "history_profit": history_profit}
# # Create the DataFrame
# df_lstm = pd.DataFrame(result_lstm)

import plotly.graph_objects as go

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_lstm["time"], y=df_lstm["closes"],
                    mode='lines',
                    name='closes'))
# Add 'buys' trace with an offset of 5 units along the y-axis
fig.add_trace(go.Scatter(x=df_lstm["time"], y=df_lstm["buys"],
                         mode='markers',
                         name='buys'))

# Add 'sells' trace with an offset of -5 units along the y-axis
fig.add_trace(go.Scatter(x=df_lstm["time"], y=df_lstm["sells"] ,
                         mode='markers',
                         name='sells'))
# Set the title
fig.update_layout(title='Stock Prices and Trading Points (Jan 2023) - BTCUSDT - Lstm-DQN',
                  xaxis=dict(title='Time'),
                  yaxis=dict(title='Price / Trading Point'))
fig.update_layout(width=800, height=400)

fig.show()

In [25]:
import plotly.graph_objects as go

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_lstm["time"], y=df_lstm["history_capital"],
                    mode='lines',
                    name='lines'))
# Set the title and axis labels
fig.update_layout(title='Capital Trend Over Time - Lstm-DQN',
                  xaxis=dict(title='Time'),
                  yaxis=dict(title='Capital'))

fig.update_layout(width=800, height=400)

fig.show()

In [26]:
import plotly.graph_objects as go

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_lstm["time"], y=df_lstm["history_profit"],
                    mode='lines',
                    name='lines'))
# Set the title
fig.update_layout(title='History Profit Over Time',
                  xaxis=dict(title='Time'),
                  yaxis=dict(title='History Profit'))

fig.show()

In [63]:
import plotly.graph_objects as go

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_lstm["time"], y=df_lstm["history_profit"],
                    mode='lines',
                    name='LSTM-DQN'))

fig.add_trace(go.Scatter(x=df_linear["time"], y=df_linear["history_profit"],
                    mode='lines',
                    name='LINEAR-DQN'))

# Set the title
fig.update_layout(title='Accumulate Profit Over Time',
                  xaxis=dict(title='Time'),
                  yaxis=dict(title='History Profit'),
                  width=800,  # Chiều rộng là 800px
                  height=400)  # Chiều cao là 400px

fig.show()

In [26]:
df_linear.describe()

,closes,buys,sells,history_capital,history_profit
count,1990.000000,559.000000,556.000000,1990.000000,1990.000000
mean,18036.606181,18001.938283,18012.596043,55894.103518,5835.411005
std,1680.515439,1687.303073,1679.652716,33283.609238,6316.166912
min,16504.200000,16517.800000,16504.200000,5.500000,-58.500000
25%,16819.900000,16814.900000,16820.475000,31958.525000,1018.300000
50%,17187.200000,16991.000000,17124.700000,53679.050000,2467.100000
75%,18990.100000,18948.750000,18931.300000,84113.800000,10926.500000
max,21314.400000,21314.400000,21294.400000,118173.900000,18194.100000


In [27]:
df_lstm.describe()

,closes,buys,sells,history_capital,history_profit
count,1990.000000,577.000000,575.000000,1990.000000,1990.000000
mean,18036.606181,18051.454593,18061.307130,53858.834372,4533.200603
std,1680.515439,1702.150222,1700.430734,32203.937124,4559.704441
min,16504.200000,16504.200000,16504.200000,105.600000,-73.000000
25%,16819.900000,16819.000000,16816.400000,28752.200000,1077.400000
50%,17187.200000,17188.100000,17177.000000,50781.700000,2577.800000
75%,18990.100000,19248.400000,19126.600000,84101.175000,7526.800000
max,21314.400000,21314.400000,21294.400000,113271.400000,13295.500000


## LSTM to compare

In [30]:
from datetime import datetime

# Unix timestamp in milliseconds
timestamp = 1609462800000

# Convert to seconds by dividing by 1000
timestamp_seconds = timestamp // 1000

# Convert to datetime
date_time = datetime.fromtimestamp(timestamp_seconds)

print(date_time)


2021-01-01 08:00:00
